# 14-day cumulative number of COVID-19 cases per 100 000

At the end of the page, we provide a detailed description of how the numbers are calculated.

### Compute data

In [1]:
import datetime as dt
import pandas as pd
pd.set_option("max_rows", None)
from oscovida import fetch_cases, get_population, get_country_data

# get a list of all country names
countries = fetch_cases().index.drop_duplicates()

data = []
yesterday = dt.date.today() - dt.timedelta(days=1) 

for region in countries:
    c, _, _ = get_country_data(region)   # get cumulative infections c
    if c.index[-1].date() < yesterday:
        print(f"{region}: last data is from {c.index[-1].date()}")
        origin = c.index[0].date()
        # Fill data series forward up to yesterday
        new_idx = pd.date_range(origin, periods=(yesterday - origin).days, freq='D')
        c.reindex(new_idx, method='pad')
    c = c[-15:]
    try:
        population = get_population(region)
        new_cases = int(c[-1] - c[-15]) 
        incidence = new_cases / population * 100000. 
        data += [(region, population, new_cases, round(incidence, 1))]
    except ValueError:
        print(f"Skip {region}")    # skip regions for which we have no population numbers

data.sort(key=lambda x: x[3], reverse=True)

# turn into pandas DataFrame for easier display
table = pd.DataFrame(data, columns=["country", "population", "new cases", "14-day-incidence"]).set_index("country")

# Show last update date
import time
print(f"Last updated {time.asctime()}")

Downloaded data: last data point 8/17/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
Downloaded data: last data point 8/17/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
Skip Diamond Princess
Skip MS Zaandam
Last updated Tue Aug 18 18:03:07 2020


## Table for all countries

In [2]:
table.sort_index()

,population,new cases,14-day-incidence
country,,,
Afghanistan,38928341,852,2.2
Albania,2877800,1879,65.3
Algeria,43851043,7053,16.1
Andorra,77265,68,88.0
Angola,32866268,655,2.0
Antigua and Barbuda,97928,1,1.0
Argentina,45195777,92383,204.4
Armenia,2963234,2599,87.7
Australia,25459700,5043,19.8


## Table sorted by 14-day-incidence

In [3]:
table

,population,new cases,14-day-incidence
country,,,
Panama,4314768,14087,326.5
Bahrain,1701583,5350,314.4
Peru,32971846,102846,311.9
Maldives,540542,1616,299.0
Colombia,50882884,148810,292.5
Brazil,212559409,609252,286.6
Israel,8655541,20321,234.8
US,329466283,724785,220.0
Argentina,45195777,92383,204.4


## Tutorial: Detailed calculation for one country

In [4]:
cases, deaths, label = get_country_data("Germany")

Cumulative cases yesterday (numbers for today are not known yet):

In [5]:
c_y = cases[-1]
c_y

226700

Cumulative cases 15 days ago (i.e. 14 days before yesterday)

In [6]:
c_15 = cases[-15]
c_15

212111

New cases from the last 14 days

In [7]:
c_new = c_y - c_15 
c_new

14589

Get the population data

In [8]:
population = get_population("Germany")
population

83128805

Compute the 14-day incidence per 100000, i.e. the cumulative number of new infections in the last 14 days, normalised by the country's population in units of 100000:

In [9]:
incidence = c_new / (population/100000)
incidence

17.54987335617299

This should compare to the numbers as they are provided for example at https://www.ecdc.europa.eu/en/cases-2019-ncov-eueea (assuming the infection numbers from ECDE and JHU are identical, and we are looking at the same dates).

---------------

In [10]:
# For debugging: When was the data downloaded
import oscovida
oscovida.fetch_cases_last_execution()

'18/08/2020 18:03:03'

In [11]:
oscovida.display_binder_link('14-day-incidence.ipynb')

[Execute this notebook with Binder](https://mybinder.org/v2/gh/oscovida/binder/master?filepath=ipynb/14-day-incidence.ipynb)